In [2]:
import pandas as pd
df=pd.read_excel('flight_price.xlsx')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [3]:
df.isnull().sum()
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [5]:
df['Date']=df['Date_of_Journey'].str.split('/').str[0]
df['Month']=df['Date_of_Journey'].str.split('/').str[1]
df['Year']=df['Date_of_Journey'].str.split('/').str[2]


In [6]:
df.drop('Date_of_Journey',axis=1,inplace=True)

In [7]:
df['Date']=df['Date'].astype('int')
df['Month']=df['Month'].astype('int')
df['Year']=df['Year'].astype('int')

In [8]:
df['departure_hours']=df['Dep_Time'].str.split(':').str[0]
df['departure_mins']=df['Dep_Time'].str.split(':').str[1]

In [9]:
df['departure_hours']=df['departure_hours'].astype('int')
df['departure_mins']=df['departure_mins'].astype('int')
df.drop('Dep_Time',axis=1,inplace=True)

In [10]:
df.drop('Route',axis=1,inplace=True)

In [11]:
df['Arrival_Time']

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [12]:
df['Arrival_hours']=df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[0]
df['Arrival_minutes']=df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[1]

In [13]:
df['Arrival_hours']=df['Arrival_hours'].astype('int')
df['Arrival_minutes']=df['Arrival_minutes'].astype('int')
df.drop('Arrival_Time',axis=1,inplace=True)

In [14]:
def convert_to_minutes(duration):
    duration = duration.strip()  # Remove leading/trailing spaces
    hours, minutes = 0, 0  # Default values

    if 'h' in duration:
        hours = int(duration.split('h')[0])  # Extract hours
        duration = duration.split('h')[1]  # Get the remaining part

    if 'm' in duration:
        minutes = int(duration.replace('m', '').strip())  # Extract minutes

    return hours * 60 + minutes  # Convert to total minutes

df['Duration_mins'] = df['Duration'].apply(convert_to_minutes)


In [15]:
df.drop('Duration',axis=1,inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Source           10683 non-null  object
 2   Destination      10683 non-null  object
 3   Total_Stops      10682 non-null  object
 4   Additional_Info  10683 non-null  object
 5   Price            10683 non-null  int64 
 6   Date             10683 non-null  int64 
 7   Month            10683 non-null  int64 
 8   Year             10683 non-null  int64 
 9   departure_hours  10683 non-null  int64 
 10  departure_mins   10683 non-null  int64 
 11  Arrival_hours    10683 non-null  int64 
 12  Arrival_minutes  10683 non-null  int64 
 13  Duration_mins    10683 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 1.1+ MB


In [17]:
df['Airline'].value_counts()

Airline
Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: count, dtype: int64

In [18]:
df['Source'].value_counts()

Source
Delhi       4537
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: count, dtype: int64

In [19]:
df['Destination'].value_counts()

Destination
Cochin       4537
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: count, dtype: int64

In [20]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
encoded=encoder.fit_transform(df[['Airline','Source','Destination']]).toarray()
encoder_df=pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

df=pd.concat([df,encoder_df],axis=1)

In [21]:
df.drop(['Airline','Source','Destination'],axis=1,inplace=True)

In [22]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Total_Stops                                10682 non-null  object 
 1   Additional_Info                            10683 non-null  object 
 2   Price                                      10683 non-null  int64  
 3   Date                                       10683 non-null  int64  
 4   Month                                      10683 non-null  int64  
 5   Year                                       10683 non-null  int64  
 6   departure_hours                            10683 non-null  int64  
 7   departure_mins                             10683 non-null  int64  
 8   Arrival_hours                              10683 non-null  int64  
 9   Arrival_minutes                            10683 non-null  int64  
 10  Duration_mins         

In [23]:
df['Total_Stops'].value_counts(dropna=False)

Total_Stops
1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
NaN            1
4 stops        1
Name: count, dtype: int64

In [24]:
df['Total_Stops'].fillna('1 stop',inplace=True)

C:\Users\vatsa\AppData\Local\Temp\ipykernel_13508\3748880211.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Total_Stops'].fillna('1 stop',inplace=True)


In [25]:
df['Total_Stops'].value_counts(dropna=False)


Total_Stops
1 stop      5626
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: count, dtype: int64

In [26]:
from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder(categories=[['non-stop','1 stop','2 stops','3 stops','4 stops']])
df['Total_Stops_encoded']=encoder.fit_transform(df[['Total_Stops']])
df.drop('Total_Stops',axis=1,inplace=True)


In [27]:
df['Total_Stops_encoded'].value_counts(dropna=False)

Total_Stops_encoded
1.0    5626
0.0    3491
2.0    1520
3.0      45
4.0       1
Name: count, dtype: int64

In [28]:
df['Additional_Info'].value_counts(dropna=False)

Additional_Info
No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
1 Short layover                    1
Red-eye flight                     1
2 Long layover                     1
Name: count, dtype: int64

In [29]:
mapping = {
    'No info': 0,
    'No Info': 0,  # Handling both variations
    'In-flight meal not included': 1,
    'No check-in baggage included': 2,
    '1 Short layover': 3,
    '1 Long layover': 4,
    '2 Long layover': 5,
    'Change airports': 6,
    'Red-eye flight': 7,
    'Business class': 8
}
df['Encoded_Info'] = df['Additional_Info'].map(mapping)


In [30]:
df.drop('Additional_Info',axis=1,inplace=True)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Price                                      10683 non-null  int64  
 1   Date                                       10683 non-null  int64  
 2   Month                                      10683 non-null  int64  
 3   Year                                       10683 non-null  int64  
 4   departure_hours                            10683 non-null  int64  
 5   departure_mins                             10683 non-null  int64  
 6   Arrival_hours                              10683 non-null  int64  
 7   Arrival_minutes                            10683 non-null  int64  
 8   Duration_mins                              10683 non-null  int64  
 9   Airline_Air Asia                           10683 non-null  float64
 10  Airline_Air India     